# My COVID-19 Dashboard

My COVID-19 Dashboard focuses on generating an interactive bar chart displaying the number of female and male cases associated with a distribution of age ranges. Extending this initial plot, I identified 3 key dates associated with various milestones of UK's COVID situation, and added interactivity in the bar chart to display the respective data up to these key dates. The 3 dates I chose to be of importance are as follows:

1) 23 March 2020 - Johnson announced the "biggest lockdown of society in British history" on national television. Social distancing was made legally mandatory and furlough scheme announced.

2) 4 January 2021 - Johnson announced new national lockdown measures for England due to the spread of another variant, later known as Alpha, that he described as “both frustrating and alarming

3) 19 July 2021 - England celebrated “Freedom Day” as the vast majority of Covid-19 restrictions were finally lifted



In [2]:
# Importing of important modules 

from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

# Matplotlib configuration 
%matplotlib inline

# make figures larger
plt.rcParams['figure.dpi'] = 100

# 

In [3]:
# Loading of canned data from JSON date files
# The variables are named with as much detail as possible for clear understanding
# Each json file contains gender and age distribution data up until the stated date

with open("england23March2020Data.json", "rt") as INFILE:
    initialEngland23March2020AgeGenderData=json.load(INFILE)
with open("england04Jan2021Data.json", "rt") as INFILE:
    initialEngland04Jan2021AgeGenderData=json.load(INFILE)
with open("england19Jul2021Data.json", "rt") as INFILE:
    initialEngland19Jul2021AgeGenderData=json.load(INFILE)

In [4]:
def wrangle_data(AgeDistributionData):
    """ Parameters: - Data imported from JSON file in above section. Returns a pandas dataframe. """
    dataDictionary=AgeDistributionData[0] # data['data'] is a list
    maleCases=dataDictionary['males']
    femaleCases=dataDictionary['females']
    ageranges=[x['age'] for x in maleCases]

    def min_age(agerange):
        """ Parameters: List of ages. Returns the minimum age of each age band"""
        agerange=agerange.replace('+','') 
        start=agerange.split('_')[0]
        return int(start)

    
    ageranges.sort(key=min_age) # sorts the age ranges in order of ascending age

    DataFrame=pd.DataFrame(index=ageranges, columns=['males','females', 'total']) # initialization of dataframe


    for entry in maleCases: # each entry is a dictionary
        ageband=entry['age'] # our index position
        DataFrame.loc[ageband, 'males']=entry['value'] # enters data into male section of dataframe
        for entry in femaleCases:
            ageband=entry['age']
            DataFrame.loc[ageband, 'females']=entry['value'] # enters data into male section of dataframe
        DataFrame['total']=DataFrame['males']+DataFrame['females'] # enters data into total section of dataframe
    return DataFrame

# each dataset corresponding to a date is wrangled and assigned to an individual variable 
england23March2020AgeGenderDataFrame=wrangle_data(initialEngland23March2020AgeGenderData)
england04Jan2021AgeGenderDataFrame=wrangle_data(initialEngland04Jan2021AgeGenderData) 
england19Jul2021AgeGenderDataFrame=wrangle_data(initialEngland19Jul2021AgeGenderData)  


In [5]:
def access_api():
    """ Accesses the PHE API. Returns raw data in the same format as data loaded from the "canned" JSON file. """   
    englandFilter = [
        'areaType=nation',
        'areaName=England'
    ]
    structure = {
        "date":"date",
        "males": "maleCases",
        "females": "femaleCases"
    }

    englandAPI = Cov19API(filters=englandFilter, structure=structure)

    updatedEnglandData=englandAPI.get_json()

    updatedEngland23March2020Data = []
    updatedEngland04Jan2021Data = []
    updatedEngland19Jul2021Data = []

    # The below for loop filters the entire date range for the individual milestone dates
    for i in range(1000):  # range is 1000 as that is approximately the number of dates accessible by the API
        if updatedEnglandData['data'][i]['date'] == "2020-03-23":
            updatedEngland23March2020Data.append(updatedEnglandData['data'][i])
        elif updatedEnglandData['data'][i]['date'] ==  "2021-01-04":
            updatedEngland04Jan2021Data.append(updatedEnglandData['data'][i])
        elif updatedEnglandData['data'][i]['date'] == "2021-07-19":
            updatedEngland19Jul2021Data.append(updatedEnglandData['data'][i])

    return [updatedEngland23March2020Data, updatedEngland04Jan2021Data, updatedEngland19Jul2021Data]

  



  

In [6]:
def api_button_callback(button):
    """ Parameters - A refresh data button. Accesses API, wrangles data, overrides global variable dataframes used for plotting. """
    try:
        englandDataInDateOrder = access_api()
    except Exception:
        print("Sorry, there was difficulty connecting to UK.GOVs Open Data API. Please try again later.")
    
    # the following variables are made global to overwrite the dataframes generated from canned data
    global england23March2020AgeGenderDataFrame 
    global england04Jan2021AgeGenderDataFrame
    global england19Jul2021AgeGenderDataFrame

    england23March2020AgeGenderDataFrame=wrangle_data(englandDataInDateOrder[0])
    england04Jan2021AgeGenderDataFrame=wrangle_data(englandDataInDateOrder[1])
    england19Jul2021AgeGenderDataFrame=wrangle_data(englandDataInDateOrder[2])

    apibutton.icon="check"
    apibutton.disabled=False


apibutton=wdg.Button(
    description='Refresh data',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to download current Public Health England data',
    icon='download' # (FontAwesome names without the `fa-` prefix)
)

apibutton.on_click(api_button_callback) # when the "apibutton" is clicked, api_button_callback function called




## Final outcome 

The following graph illustrates the relationship plots the number of male cases and female cases within certain age ranges. This graph has multiple widgets, positioned on the right side of the graph. The topmost widget, "Gender", illustrates only the cases associated with the respective gender. The lower widget, "Date", illustrates the cumulative data up to that given date.

In [16]:
agecols=wdg.SelectMultiple(
    options=['males', 'females', 'total'], # options available
    value=['males', 'females'], # initial values selected
    rows=3, # rows of the selection box
    description='Gender',
    disabled=False
)

date=wdg.RadioButtons(
    options=['23rd March 2020',
            '4th January 2021', 
            '19th July 2021'], # available options
    value='23rd March 2020', # initial value selected
    description='Date:',
    disabled=False
)

def age_graph(graphcolumns, datecolumns):
    """ Parameters - widgets defined above. Function - Chooses the data used to plot the graph depending on the widgets toggled"""
    
    ncols=len(graphcolumns)
    if datecolumns == '23rd March 2020':
        if ncols>0:
            england23March2020AgeGenderDataFrame.plot(kind='bar', y=list(graphcolumns)) # graphcolumns is a tuple - we need a list
            plt.title("A comparison of confirmed covid cases between males, females, and different age ranges.", size = 16, pad=30)
            plt.ylabel("Number of confirmed Covid Cases", size = 14)
            plt.xlabel("Age ranges", size = 14)
            plt.show()
        else:
            # if the user has not selected any column, print a message instead
            print("Click to select data for graph")
            print("(CTRL-Click to select more than one category)")
    if datecolumns == '4th January 2021':
        if ncols>0:
            england04Jan2021AgeGenderDataFrame.plot(kind='bar', y=list(graphcolumns)) # graphcolumns is a tuple - we need a list
            plt.title("A comparison of confirmed covid cases between males, females, and different age ranges.")
            plt.ylabel("Number of confirmed Covid Cases")
            plt.xlabel("Age ranges")
            plt.show() 
         
        else:
            # if the user has not selected any column, print a message instead
            print("Click to select data for graph")
            print("(CTRL-Click to select more than one category)")
    if datecolumns == '19th July 2021':
        if ncols>0:
            england19Jul2021AgeGenderDataFrame.plot(kind='bar', y=list(graphcolumns)) # graphcolumns is a tuple - we need a list
            plt.title("A comparison of confirmed covid cases between males, females, and different age ranges.")
            plt.ylabel("Number of confirmed Covid Cases")
            plt.xlabel("Age ranges")
            plt.show() 
        else:
            # if the user has not selected any column, print a message instead
            print("Click to select data for graph")
            print("(CTRL-Click to select more than one category)")
    
output=wdg.interactive_output(age_graph, {'graphcolumns': agecols, 'datecolumns': date})

ctrls = wdg.VBox([agecols, date])
form=wdg.HBox([output, ctrls])

display(form)






# Instructions on how to interact with graph

There are 3 options for the gender widget: "males", "females", "total". The "male" option will plot data for male confirmed covid cases. The "female" option will plot data for female confirmed covid cases. The "total" option will plot data for the total cases associated with that age range irrespective of gender. Another feature of this widget, is that multiple options can be selected simultaneously. On selection of more than 1 option, the selected options will be simultaneously displayed on the graph, allowing you to draw clear comparison between data for different genders. There are 3 options for selecting the date widget. On selecting the option 23rd March 2020 the total male cases, total female cases, and total overall cases up until that date are used as data for plotting the graphs. For 4th January 2021, the data is extending to include additional cases between 23rd March 2020 and the 4th January 2021. Same for 19th July 2021. 

# Refreshing the data

The data immedietly loaded is "canned" data stored in an offline JSON file as a secure form of storage. If you would like up to date information, press the "Refresh data" button below. (Note: The graph will not refresh until a widget is interacted with)

In [17]:
display(apibutton)

Button(description='Refresh data', icon='check', style=ButtonStyle(), tooltip='Click to download current Publi…

# Brief Conclusions

Several brief conclusions can be drawn from playing with the graph. Firstly, there is a significant difference in the age distribution for the different dates. It can be seen that data up until the 23rd March 2020, shows a left tailed pattern with more cases lying in higher age ranges taken precedence over lower age ranges. Through time, this pattern is reversed and lower age ranges take majority of cases resulting in a right tailed distribution. 

**Author and Copyright Notice** Covid-19 Dashboard (C) Nicolas Robinson, 2022 (ec221130@qmul.ac.uk - web). All rights reserved.